In [1]:
import pandas as pd
import evidently
print(evidently.__version__)
import numpy as np
from IPython.display import display, Markdown

0.7.18


In [2]:
from rich import print as rprint
from rich.tree import Tree

def to_tree(obj, name="root"):
    """
    Recursively convert dicts, lists, and objects with __dict__ to a rich Tree.
    """
    tree = Tree(f":star: [bold]{name}[/bold]")
    
    if isinstance(obj, dict):
        items = obj.items()
    elif hasattr(obj, "__dict__"):
        items = vars(obj).items()
    else:
        # fallback for other types
        tree.add(f"[cyan]{obj}[/cyan]")
        return tree
    
    for k, v in items:
        if isinstance(v, (dict, object)) and not isinstance(v, (str, int, float, bool, type(None))):
            tree.add(to_tree(v, name=str(k)))
        else:
            tree.add(f"[yellow]{k}[/yellow]: [cyan]{v}[/cyan]")
    return tree


## Load Dataset and MLFlow Runs

In [20]:
dfRaw = pd.read_csv(r"/home/srirama/sr_proj/EmotionAnalysis/data/Twitter/raw/train.csv",header=None)
dfProc = pd.read_csv(r"/home/srirama/sr_proj/EmotionAnalysis/src/artifacts/data/preprocessed/train.csv")

display(Markdown("### 🔹 Raw Twitter Data"))
display(dfRaw.head())

display(Markdown("### 🔹 Preprocessed Data"))
display(dfProc.head())

### 🔹 Raw Twitter Data

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


### 🔹 Preprocessed Data

,sentiment,cleaned_text
0,Positive,geting borderlands day wil murder al
1,Positive,coming borders day wil kil al
2,Positive,geting borderlands day wil kil al
3,Positive,coming borderlands day wil murder al
4,Positive,geting borderlands day wil murder al


In [4]:
ref_ratio = 0.6
split_idx = int(len(dfProc) * ref_ratio)
refDfProc = dfProc.iloc[:split_idx].reset_index(drop=True)
currDfProc = dfProc.iloc[split_idx:].reset_index(drop=True)

In [5]:
import mlflow
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri("http://localhost:5000")

In [6]:
client = MlflowClient()
experiment_name = "1-REG"
experiment = client.get_experiment_by_name(experiment_name)
runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        filter_string="attributes.status = 'FINISHED'",
        order_by=["attributes.start_time DESC"],
        # max_results=1,
    )
rprint(to_tree(experiment, name=f"Experiment : {experiment_name}"))
latest_run = runs[0].to_dictionary()
rprint(to_tree(latest_run, name="Latest Run"))

⭐ Experiment : 1-REG
├── _experiment_id: 1
├── _name: 1-REG
├── _artifact_location: mlflow-artifacts:/1
├── _lifecycle_stage: active
├── ⭐ _tags
│   └── mlflow.experimentKind: custom_model_development
├── _creation_time: 1766845331089
└── _last_update_time: 1766845331089

⭐ Latest Run
├── ⭐ info
│   ├── artifact_uri: mlflow-artifacts:/1/7493f00dd1d74d4a9632b0432cf66fc9/artifacts
│   ├── end_time: 1766845420454
│   ├── experiment_id: 1
│   ├── lifecycle_stage: active
│   ├── run_id: 7493f00dd1d74d4a9632b0432cf66fc9
│   ├── run_name: rnn_1
│   ├── start_time: 1766845331314
│   ├── status: FINISHED
│   └── user_id: srirama
├── ⭐ data
│   ├── ⭐ metrics
│   │   ├── validation_accuracy: 0.3031262457370758
│   │   ├── validation_loss: 1.372296929359436
│   │   ├── accuracy: 0.2995634973049164
│   │   ├── loss: 1.3738007545471191
│   │   ├── val_accuracy: 0.3031262457370758
│   │   └── val_loss: 1.372296929359436
│   ├── ⭐ params
│   │   ├── epochs: 1
│   │   ├── batch_size: 400
│   │   ├── validation_split: 0.0
│   │   ├── shuffle: True
│   │   ├── class_weight: None
│   │   ├── sample_weight: None
│   │   ├── initial_epoch: 0
│   │   ├── steps_per_epoch: None
│   │   ├── validation_steps: None
│   │   ├── validation_batch_size: None
│   │   ├── validation_freq: 1
│   │   ├── opt_name: adam
│   │   ├── opt_learning_rate: 0.0010000000474974513
│   │   ├── opt_weight_decay: None
│   │   ├── opt_clipnorm: None
│   │   ├── opt_global_clipnorm: None
│   │   ├── opt_clipvalue: None
│   │   ├── opt_use_ema: False
│   │   ├── opt_ema_momentum: 0.99
│   │   ├── opt_ema_overwrite_frequency: None
│   │   ├── opt_loss_scale_factor: None
│   │   ├── opt_gradient_accumulation_steps: None
│   │   ├── opt_beta_1: 0.9
│   │   ├── opt_beta_2: 0.999
│   │   ├── opt_epsilon: 1e-07
│   │   └── opt_amsgrad: False
│   └── ⭐ tags
│       ├── mlflow.user: srirama
│       ├── mlflow.source.name: src/train/user_input.py
│       ├── mlflow.source.type: LOCAL
│       ├── mlflow.source.git.commit: c78a81d99de035e28f20aceb6d5457595e5ceadd
│       ├── mlflow.runName: rnn_1
│       └── mlflow.latest_checkpoint_artifact: checkpoints/latest_checkpoint.h5
├── ⭐ inputs
│   ├── ⭐ model_inputs
│   │   └── []
│   └── ⭐ dataset_inputs
│       └── [{'dataset': {'name': 'dataset', 'digest': '51d321f3', 'source_type': 'code', 'source': '{"tags": 
│           {"mlflow.user": "srirama", "mlflow.source.name": "src/train/user_input.py", "mlflow.source.type": 
│           "LOCAL", "mlflow.source.git.commit": "c78a81d99de035e28f20aceb6d5457595e5ceadd"}}', 'schema': 
│           '{"mlflow_tensorspec": {"features": "[{\\"type\\": \\"tensor\\", \\"tensor-spec\\": {\\"dtype\\": 
│           \\"int32\\", \\"shape\\": [-1, 75]}}]", "targets": "[{\\"type\\": \\"tensor\\", \\"tensor-spec\\": 
│           {\\"dtype\\": \\"float64\\", \\"shape\\": [-1, 4]}}]"}}', 'profile': '{"features_shape": [41694, 75], 
│           "features_size": 3127050, "features_nbytes": 12508200, "targets_shape": [41694, 4], "targets_size": 
│           166776, "targets_nbytes": 1334208}'}, 'tags': {'mlflow.data.context': 'train'}}, {'dataset': {'name': 
│           'dataset', 'digest': '71ada469', 'source_type': 'code', 'source': '{"tags": {"mlflow.user": "srirama", 
│           "mlflow.source.name": "src/train/user_input.py", "mlflow.source.type": "LOCAL", 
│           "mlflow.source.git.commit": "c78a81d99de035e28f20aceb6d5457595e5ceadd"}}', 'schema': 
│           '{"mlflow_tensorspec": {"features": "[{\\"type\\": \\"tensor\\", \\"tensor-spec\\": {\\"dtype\\": 
│           \\"int32\\", \\"shape\\": [-1, 75]}}]", "targets": "[{\\"type\\": \\"tensor\\", \\"tensor-spec\\": 
│           {\\"dtype\\": \\"float64\\", \\"shape\\": [-1, 4]}}]"}}', 'profile': '{"features_shape": [27797, 75], 
│           "features_size": 2084775, "features_nbytes": 8339100, "targets_shape": [27797, 4], "targets_size": 
│           111188, "targets_nbytes": 889504}'}, 'tags': {'mlflow.data.context': 'eval'}}]
└── ⭐ outputs
    └── ⭐ model_outputs
        └── [{'model_id': 'm-05de093039d84da587df488841cdd343', 'step': 0}]

## Load Model for prediction

In [7]:
model = mlflow.tensorflow.load_model(f'mlflow-artifacts:/{experiment.experiment_id}/models/{latest_run["outputs"]["model_outputs"][0]["model_id"]}/artifacts') # Works
# model = mlflow.tensorflow.load_model("/home/srirama/sr_proj/EmotionAnalysis/mlartifacts/1/models/m-05de093039d84da587df488841cdd343/artifacts") # More direct method
model.summary()

2025-12-28 16:52:53.821153: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/srirama/sr_proj/tf-venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
/home/srirama/sr_proj/tf-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1766940781.079142    1817 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 75, 12)         │       268,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 256)            │        68,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │            20 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 762,418 (2.91 MB)

 Trainable params: 381,208 (1.45 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 381,210 (1.45 MB)

## Preprocessing using tokenizer

In [8]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

def load_tokenizer(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        tok_dict = json.load(f)
    return tokenizer_from_json(tok_dict)

TOKENIZER_JSON_FILE = "/home/srirama/sr_proj/EmotionAnalysis/src/artifacts/tokenizer.json"

In [10]:
currDfProc['cleaned_text'].apply(type).value_counts()

cleaned_text
<class 'str'>      27564
<class 'float'>      233
Name: count, dtype: int64

In [11]:
currDfProc['cleaned_text'] = (
    currDfProc['cleaned_text']
    .fillna("")          # replace NaN
    .astype(str)         # force string
)

In [12]:
tokenizer = load_tokenizer(str(TOKENIZER_JSON_FILE))
sequences  = tokenizer.texts_to_sequences(currDfProc['cleaned_text'])
print("Total Sequences",len(sequences) )# = total rows
print("Length of 1st sequence",len(sequences[0])) # = 1st row word count
maxlen = 75
padded_sentence_sequences = pad_sequences(sequences, maxlen = maxlen, padding='pre')
print("Padded Sequence shape :",padded_sentence_sequences.shape)
padded_sentence_sequences

Total Sequences 27797
Length of 1st sequence 20
Padded Sequence shape : (27797, 75)


array([[   0,    0,    0, ...,  758,  186,  730],
       [   0,    0,    0, ...,  758,  186,  730],
       [   0,    0,    0, ...,    3,    4,  953],
       ...,
       [   0,    0,    0, ...,    2,  371, 1778],
       [   0,    0,    0, ...,  371,   64, 1778],
       [   0,    0,    0, ...,    6,  371, 1778]],
      shape=(27797, 75), dtype=int32)

In [15]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
label_encoder.fit(dfProc['sentiment'])
y = to_categorical(label_encoder.transform(currDfProc["sentiment"]))  # one-hot encode for multi-class
y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], shape=(27797, 4))

In [16]:
y_pred  = model.predict(padded_sentence_sequences)

2025-12-28 16:56:00.656715: I external/local_xla/xla/service/service.cc:163] XLA service 0x11e39340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-28 16:56:00.656790: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 2050, Compute Capability 8.6
2025-12-28 16:56:00.720469: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-28 16:56:00.869537: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701
2025-12-28 16:56:00.917822: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-28 16:56:02.714766: I exter

 20/869 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step  

I0000 00:00:1766940964.835958    4141 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


866/869 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

2025-12-28 16:56:11.020972: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-28 16:56:12.719582: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_206', 292 bytes spill stores, 292 bytes spill loads



869/869 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step


## Using Evidently AI

In [44]:
# ! pip install ipynb-py-convert
! ipynb-py-convert 6-evidently-ai.ipynb 6-evidently-ai.py
# ! pip install -U evidently

In [22]:
import evidently
print("Evidently version:", evidently.__version__)

Evidently version: 0.7.18


In [47]:
import pandas as pd
import numpy as np

from evidently import Report
from evidently.metrics import *
from evidently.presets import *

In [27]:
# Convert one-hot predictions to class labels
y_pred_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))
currDfProc = currDfProc.copy()
currDfProc["prediction"] = y_pred_labels
currDfProc.sample(3)

,sentiment,cleaned_text,prediction
2536,Neutral,day,Negative
5898,Positive,fifa fun fr video comandes,Negative
11158,Negative,got hit min bc abandon sanction play got disco...,Negative


In [28]:
refDfProc = refDfProc.copy()
refDfProc["prediction"] = refDfProc["sentiment"]
refDfProc.head(3)

,sentiment,cleaned_text,prediction
0,Positive,geting borderlands day wil murder al,Positive
1,Positive,coming borders day wil kil al,Positive
2,Positive,geting borderlands day wil kil al,Positive


In [46]:
report = Report(metrics=[
    DataSummaryPreset(),
    DataDriftPreset(),
    ClassificationPreset(),

])

snippet = report.run(
    reference_data=refDfProc,
    current_data=currDfProc,
)

snippet.save_html("sentiment_monitoring.html")


ValueError: Cannot use ClassificationPreset without a classification configration